In [1]:
from torch import  nn
import torch
from torch import optim
from mlp import MLP

### nn.Model 类

In [2]:
from perception_sequential import Perception


In [3]:
model = Perception(100,10000,10).cuda()
model


Perception(
  (layer): Sequential(
    (0): Linear(in_features=100, out_features=10000, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=10000, out_features=10, bias=True)
    (3): Sigmoid()
  )
)

In [4]:
input = torch.randn(100).cuda()
output = model(input)
output.shape

torch.Size([10])

## 模型处理
- 2.4.1 网络模型库： torchvision.models (包含以下经典的网络结构和预训练模型）
    - VGG
    - ResNet
    - Inception 等

In [6]:
from torch import nn
from torchvision import models

# 通过torchvision.model 直接调用VGG16的网络结构
vgg = models.vgg16()
# VGG16 的特征层包含13个卷积、13个激活函数ReLu，5个池化，一共31层.注意：这个是特征层
len(vgg.features)

31

In [9]:
# VGG16d 包含3个全连接、2个ReLU、2个Dropout，一共7层
len(vgg.classifier)

7

In [10]:
# 可以通过出现的顺序直接索引每一层
vgg.classifier[-1]

Linear(in_features=4096, out_features=1000, bias=True)

In [13]:
# 也可以选取某一部分，如下代表了特征网络的最后一个卷积模组
vgg.features[24:]

Sequential(
  (24): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (25): ReLU(inplace=True)
  (26): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (27): ReLU(inplace=True)
  (28): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (29): ReLU(inplace=True)
  (30): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

### 2.4.2 加载预训练模型
- 预训练模型的来因：
    - 对于计算机视觉的任务，包括物体检测，我们通常很难拿到很大的数据集，在这种情况下重新训练一个新的模型是比较复杂的，并且不容易调整
    - 因此，Fine-tune（微调）是一个常用的选择。所谓Fine-tune是指利用别人在一些数据集上训练好的预训练模型，在自己的数据集上训练自己的模型。

In [ ]:
# 具体代码
# 第一种： 直接调用别人的预训练模型
vgg = models.vgg16(pretrained=True) # 通过torchvision.vgg16 直接叼用VGG16的网络结构

# 第二种： 叼用自己的本地预训练模型，或是之前训练过的模型
vgg = models.vgg16()
state_dict = torch.load("Your model path")
# 利用load_state_dict,遍历训练模型的关键字，如果出现在了VGG中，则加载预训练参数
# vgg.load_state_dict({k:v for k, v in state_dict_items() if k in vgg.state_dict()})

# 
# 通常来讲，对于不同的检测任务，卷积网络的前两三层的作用是非常类似的，都是提取图像的边缘信息等，因此为了保证模型训练中能够更加稳定，一般会固定预训练网络的前两三个卷积层而不进行参数的学习。例如VGG模型，可以设置前三个卷积模组不进行参数学习，设置方式如下：
# 这里对应的10就包含了前面3个卷积层
# for layer in range(10):
#     for p in vgg[layer].parameters():
#         p.requires_grad = False

### 2.4.3 模型保存
- 代码仅作为说明，不能用于运行


In [1]:
# torch.save({
#     'model' : model.state_dict(),
#     'optimizer:' : optimizer.state_dict(),
#     'model_path' : "Your model path"
# })

## 2.5 数据处理
- 2.5.1 主流公开数据集
    - ImageNet
    - PASCAL VOC 
    - COCO ( Common Objects in Context)

### 2.5.2 数据加载
- 三个步骤
    - 1)继承Dataset类
    - 2)增加数据变换
    - 3)继承Dataloader


In [ ]:
from torch.utils.data import Dataset
# 不可运行代码
# 用法：
# 1) 继承Dataset类
class my_data(Dataset):
    # 初始化读取数据集
    def __init__(self,image_path,annotation_path,transform=None):
        pass
    
    
    def __len__(self): #获取数据集的总大小
        pass
    
    def __getitem__(self, id): # 对于指定的id，读取该数据并返回
        pass    
    
# 实例化并开始遍历    
dataset = my_data("your image path","your annotation path")
for data in dataset:
    print(data)
        
    

In [5]:
# 2)数据变换和增强 torchvision.transforms. 示意代码，不可运行
from torchvision import transforms
# 将transforms集成到Dataset类中，使用Compose将多个变换整合到一起
dataset = my_data("your image path", "your annotation path", transform=transforms.Compose([
    transforms.Resize(256), # 将图像最短边缩小至256，宽高比例不变
    # 以0.5的概率随机翻转指定的PIL图像
    transforms.RandomHorizontalFlip(),
    # 将PIL图像转为Tensor，元素区间从[0, 255]归一到[0, 1]
    transforms.ToTensor(),
    # 进行mean与std为0.5的标准化
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
]))



TypeError: my_data.__init__() got an unexpected keyword argument 'transforms'

In [ ]:
# 3) 继承dataloader
# 前面两部已经可以获取每一个变化后的样本，但是仍然无法进行批量处理，随机选取等操作，因为仍需torch.utils.data.Dataloader类进一步封装
# 该类需要4个参数：
# 1 继承了Dataset的实例
# 2 批量batch的大小
# 3 是否打乱参数数据
# 4 使用几个线程来加载数据

from torch.utils.data import DataLoader
# 使用Dataloader进一步封装Dataset
dataloader = DataLoader(dataset,batch_size=4,shuffle=True,num_workers=4) # 
# 注意：dataloader是一个可迭代对象，对该实例进行迭代即可用于训练过程，其实就是训练实质.
data_iter = iter(dataloader)
for step in range(iters_per_epoch): # iters_per_epoch 代表每个批次迭代多少次
    data = next(data_iter)
    




## GPU 加速
- 判断是否可以在GPU上面进行张量运算： torch.cuda.is_available() 

In [21]:
import torch
from torchvision import models
a = torch.randn(3,3)
b = models.vgg16()

# Check current cpu is available or not
if torch.cuda.is_available():
    a = a.cuda()
    # 指定将b转移到编号为1的GPU上
    #b = b.cuda(1) 这个没有通过
    
    # 使用torch.device()来指定使用哪一个GPU
    device = torch.device("cuda:2") # 这个成功了
    device_name =torch.cuda.get_device_name(0)
    count = torch.cuda.get_device_capability(0) # 获得GPU最大和最小的CUDA计算能力
    print(count)

    

(8, 6)


## 训练可视化工具
- TensorBoardX 
- Visdom 这里只关注Facebook开发的Visdom
    - 1) 开启visdom服务- python3 -m visdom.server
      2) 打开web，网址：http://localhost:8097

In [25]:
import visdom
vis = visdom.Visdom(env='first item')
vis.text('first visdom',win='I am here')
vis.image(torch.randn(3,256,256),win='image_random')

Setting up a new session...


'image_random'